In [2]:
!pip install tensorflow

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy

import re



from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords



from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix

import gensim.downloader as api


import nltk
nltk.download('stopwords')


import gensim
import itertools


from gensim.models import KeyedVectors
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
from tensorflow.keras.preprocessing.sequence import pad_sequences






import os
import matplotlib

import tensorflow as tf

from time import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam  # Corrected import
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Embedding, LSTM

matplotlib.use('Agg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:

def text_to_word_list(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.split()

    return text

def make_w2v_embeddings(df, embedding_dim=300):
    vocabs = {}
    vocabs_cnt = 0
    vocabs_not_w2v = {}
    vocabs_not_w2v_cnt = 0

    stops = set(stopwords.words('english'))



    word2vec_model_name = 'word2vec-google-news-300'
    word2vec = api.load(word2vec_model_name)

    for index, row in df.iterrows():
        if index != 0 and index % 1000 == 0:
            print("{:,} sentences embedded.".format(index), flush=True)

        for question in ['question1', 'question2']:

            q2n = []
            for word in text_to_word_list(row[question]):
                if word in stops:
                    continue
                if word not in word2vec.key_to_index:
                    if word not in vocabs_not_w2v:
                        vocabs_not_w2v_cnt += 1
                        vocabs_not_w2v[word] = 1
                if word not in vocabs:
                    vocabs_cnt += 1
                    vocabs[word] = vocabs_cnt
                    q2n.append(vocabs_cnt)
                else:
                    q2n.append(vocabs[word])

            df.at[index, question + '_n'] = q2n

    embeddings = 1 * np.random.randn(len(vocabs) + 1, embedding_dim)
    embeddings[0] = 0

    for word, index in vocabs.items():
        if word in word2vec.key_to_index:
            embeddings[index] = word2vec[word]

    del word2vec

    return df, embeddings


def split_and_zero_padding(df, max_seq_length):
    X = {'q1': df['question1_n'], 'q2': df['question2_n']}

    for dataset, side in itertools.product([X], ['q1', 'q2']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)

    return dataset





In [11]:
max_seq_length = 25

embedding_dim =300

In [12]:
df_pred = pd.DataFrame(np.c_[['Do you believe there is life after death?'] , ['Is it true that there is life after death?']] , columns = ['question1' , 'question2'])



for q in ['question1', 'question2']:
    df_pred[q + '_n'] = df_pred[q]

df_q2n, embeddings = make_w2v_embeddings(df_pred, embedding_dim=embedding_dim)

X = df_q2n[['question1_n', 'question2_n']]

X_pad = split_and_zero_padding(X, max_seq_length)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [13]:
model_path = '/content/drive/MyDrive/Colab_Notebooks/Projects/Similarity_Questions/siamese-lstm-weights.h5'

# Chargez le modèle
loaded_model = tf.keras.models.load_model(model_path)

prediction = loaded_model.predict([X_pad['q1'], X_pad['q2']])


1/1 [==============================] - 2s 2s/step


In [14]:
if float(prediction > 0.5) :
  print(f'The two questions are similar.')
else :
  print('There is No similarity')

The two questions are similar.


In [15]:
df_pred_2 = pd.DataFrame(np.c_[['How to learn python?'] , ['how to create the code in java?']] , columns = ['question1' , 'question2'])



for q in ['question1', 'question2']:
    df_pred_2[q + '_n'] = df_pred_2[q]

df_q2n, embeddings = make_w2v_embeddings(df_pred_2, embedding_dim=embedding_dim)

X_2 = df_q2n[['question1_n', 'question2_n']]

X_pad_2 = split_and_zero_padding(X_2, max_seq_length)

In [16]:

prediction_2 = loaded_model.predict([X_pad_2['q1'], X_pad_2['q2']])

if float(prediction_2 > 0.5) :
  print(f'The two questions are similar.')
else :
  print('There is No similarity')

1/1 [==============================] - 0s 34ms/step
There is No similarity
